# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


Analysis of variance 
when we want to compare more than two groups at the same time

In [78]:
import pandas as pd # for data manipulation
import wbgapi as wb
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.linear_model import LogisticRegression, LinearRegression # for building a logistic and linear regression model
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor

from statsmodels.formula.api import logit

Sadece fonksiyon oluşturma kısmında arkadaşımdan yardım aldım ama onun dışında kendim yapmaya çalıştım

In [3]:
def pull_data(indicator):
    #return pd.DataFrame(list(wb.data.fetch(indicator)))
    return wb.data.DataFrame(indicator, "TUR").T # Data for Turkey

indicators = { "literacy_rate" : 'SE.ADT.LITR.FE.ZS',
                "labor_force" : 'SL.TLF.TOTL.FE.ZS',
                "poverty_hc_ratio" : 'SI.POV.NAHC',
                "c_health_exp" : 'SH.XPD.CHEX.PC.CD',
                "gdp" : 'NY.GDP.PCAP.CD',
                "mortality_rate" : 'SH.DYN.MORT'              
                }
                
df = pull_data(indicators.values())
df

series,NY.GDP.PCAP.CD,SE.ADT.LITR.FE.ZS,SH.DYN.MORT,SH.XPD.CHEX.PC.CD,SI.POV.NAHC,SL.TLF.TOTL.FE.ZS
YR1960,509.005545,NaN,257.0,NaN,NaN,NaN
YR1961,283.828284,NaN,249.3,NaN,NaN,NaN
YR1962,309.446624,NaN,241.4,NaN,NaN,NaN
YR1963,350.662985,NaN,233.5,NaN,NaN,NaN
YR1964,369.583469,NaN,225.7,NaN,NaN,NaN
...,...,...,...,...,...,...
YR2017,10589.667725,93.498268,11.4,442.617615,13.9,32.799757
YR2018,9454.348443,NaN,10.7,389.865570,14.4,33.089766
YR2019,9121.515167,94.424042,10.1,396.466827,15.0,33.360649
YR2020,8536.433320,NaN,9.5,NaN,NaN,32.175606


In [4]:
df.rename(columns = {'SE.ADT.LITR.FE.ZS' : "literacy_rate",
                    'SL.TLF.TOTL.FE.ZS' : "labor_force",
                    'SI.POV.NAHC' : "poverty_hc_ratio",
                    'SH.XPD.CHEX.PC.CD' : "c_health_exp",
                    'NY.GDP.PCAP.CD' : "gdp",
                    'SH.DYN.MORT' : "mortality_rate"             
                }, inplace = True)

df

series,gdp,literacy_rate,mortality_rate,c_health_exp,poverty_hc_ratio,labor_force
YR1960,509.005545,NaN,257.0,NaN,NaN,NaN
YR1961,283.828284,NaN,249.3,NaN,NaN,NaN
YR1962,309.446624,NaN,241.4,NaN,NaN,NaN
YR1963,350.662985,NaN,233.5,NaN,NaN,NaN
YR1964,369.583469,NaN,225.7,NaN,NaN,NaN
...,...,...,...,...,...,...
YR2017,10589.667725,93.498268,11.4,442.617615,13.9,32.799757
YR2018,9454.348443,NaN,10.7,389.865570,14.4,33.089766
YR2019,9121.515167,94.424042,10.1,396.466827,15.0,33.360649
YR2020,8536.433320,NaN,9.5,NaN,NaN,32.175606


In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, YR1960 to YR2021
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gdp               62 non-null     float64
 1   literacy_rate     18 non-null     float64
 2   mortality_rate    61 non-null     float64
 3   c_health_exp      20 non-null     float64
 4   poverty_hc_ratio  16 non-null     float64
 5   labor_force       32 non-null     float64
dtypes: float64(6)
memory usage: 3.4+ KB


We can see that all of them are float

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
series,,,,,,,,
gdp,62.0,4147.598224,4008.259362,283.828284,1166.473636,2503.022470,7940.466718,12614.781610
literacy_rate,18.0,81.302351,15.052583,45.098919,79.782557,86.710167,92.335882,94.424042
mortality_rate,61.0,95.937705,76.728989,9.500000,26.200000,74.200000,158.100000,257.000000
c_health_exp,20.0,413.245073,132.304052,153.622742,348.571198,447.867050,524.648911,570.858948
poverty_hc_ratio,16.0,16.175000,2.717474,13.500000,14.625000,15.550000,16.750000,25.000000
labor_force,32.0,29.584970,2.195023,25.921956,28.230122,29.536818,31.096678,33.360649


In [7]:
pd.isnull(df.literacy_rate).sum() # also other indicators has a lot of nans.

44

As we can see from the table above, we have a lot of nans, to get a model i have fill nans with 0 first.

Even though not all independent variables seems good indicator for our dependent variable (child mortality), let's see what is R^2$-score of the model.

In [8]:
df = df.fillna(0) 
X = df[["literacy_rate", "labor_force", "poverty_hc_ratio", "c_health_exp", "gdp"]]
Y = df["mortality_rate"]
XX = sm.add_constant(X)
model = sm.OLS(Y, XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         mortality_rate   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     42.10
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           9.07e-18
Time:                        18:02:57   Log-Likelihood:                -308.48
No. Observations:                  62   AIC:                             629.0
Df Residuals:                      56   BIC:                             641.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              172.3986      7.569  

When we look at R^2, it is 0.79 which is not good. Also p - values, especially for literacy_rate and poverty_hc_ratio is not good. Let's try what will happen, when we don't use them in our model.

In [9]:
X = df[["labor_force", "c_health_exp", "gdp"]]
Y = df["mortality_rate"]
XX = sm.add_constant(X)
model = sm.OLS(Y, XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         mortality_rate   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     72.56
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           1.28e-19
Time:                        18:03:06   Log-Likelihood:                -308.52
No. Observations:                  62   AIC:                             625.0
Df Residuals:                      58   BIC:                             633.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          172.3006      7.422     23.216   

Our R^2 is still 0.79 so let's try to multiply some variables.

In [10]:
df["multiply"] = df["poverty_hc_ratio"] * df["literacy_rate"]
df["multiply2"] = df["labor_force"] * df["gdp"]

# "literacy_rate", "labor_force", "poverty_hc_ratio", "c_health_exp", "gdp", "mortality_rate"   

In [11]:
X = df[["c_health_exp", "gdp", "labor_force", 'multiply', "multiply2"]]
Y = df["mortality_rate"]
XX = sm.add_constant(X)
model = sm.OLS(Y, XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         mortality_rate   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     105.4
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           3.30e-27
Time:                        18:03:14   Log-Likelihood:                -284.20
No. Observations:                  62   AIC:                             580.4
Df Residuals:                      56   BIC:                             593.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          224.2218      8.161     27.475   

Even though the R^2 is 0,904, better than 0.767, it still doesn't feel quite right. Maybe fillna(0) part was not a good idea, instead of filling nans we can drop nan values. For this, let's run the codes again.

In [12]:
def pull_data(indicator):
    #return pd.DataFrame(list(wb.data.fetch(indicator)))
    return wb.data.DataFrame(indicator, "TUR").T # Data for Turkey

indicators = { "literacy_rate" : 'SE.ADT.LITR.FE.ZS',
                "labor_force" : 'SL.TLF.TOTL.FE.ZS',
                "poverty_hc_ratio" : 'SI.POV.NAHC',
                "c_health_exp" : 'SH.XPD.CHEX.PC.CD',
                "gdp" : 'NY.GDP.PCAP.CD',
                "mortality_rate" : 'SH.DYN.MORT'              
                }
                
df = pull_data(indicators.values())

In [13]:
df.rename(columns = {'SE.ADT.LITR.FE.ZS' : "literacy_rate",
                    'SL.TLF.TOTL.FE.ZS' : "labor_force",
                    'SI.POV.NAHC' : "poverty_hc_ratio",
                    'SH.XPD.CHEX.PC.CD' : "c_health_exp",
                    'NY.GDP.PCAP.CD' : "gdp",
                    'SH.DYN.MORT' : "mortality_rate"             
                }, inplace = True)

df

series,gdp,literacy_rate,mortality_rate,c_health_exp,poverty_hc_ratio,labor_force
YR1960,509.005545,NaN,257.0,NaN,NaN,NaN
YR1961,283.828284,NaN,249.3,NaN,NaN,NaN
YR1962,309.446624,NaN,241.4,NaN,NaN,NaN
YR1963,350.662985,NaN,233.5,NaN,NaN,NaN
YR1964,369.583469,NaN,225.7,NaN,NaN,NaN
...,...,...,...,...,...,...
YR2017,10589.667725,93.498268,11.4,442.617615,13.9,32.799757
YR2018,9454.348443,NaN,10.7,389.865570,14.4,33.089766
YR2019,9121.515167,94.424042,10.1,396.466827,15.0,33.360649
YR2020,8536.433320,NaN,9.5,NaN,NaN,32.175606


In [14]:
df = df.dropna() 
X = df[["literacy_rate", "labor_force", "poverty_hc_ratio", "c_health_exp", "gdp"]]
Y = df["mortality_rate"]
XX = sm.add_constant(X)
model = sm.OLS(Y, XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         mortality_rate   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     169.6
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           3.81e-07
Time:                        18:03:40   Log-Likelihood:                -7.8492
No. Observations:                  13   AIC:                             27.70
Df Residuals:                       7   BIC:                             31.09
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               69.8438      9.987  

c:\Users\duygu\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


0.992 R^2 seems so high, so there should be some multicollinearity, so let's check correlations first

In [15]:
df.corr()

series,gdp,literacy_rate,mortality_rate,c_health_exp,poverty_hc_ratio,labor_force
series,,,,,,
gdp,1.000000,0.708098,-0.624924,0.786933,-0.642485,0.591484
literacy_rate,0.708098,1.000000,-0.976415,0.240982,-0.849316,0.981657
mortality_rate,-0.624924,-0.976415,1.000000,-0.173542,0.885762,-0.989861
c_health_exp,0.786933,0.240982,-0.173542,1.000000,-0.192544,0.097192
poverty_hc_ratio,-0.642485,-0.849316,0.885762,-0.192544,1.000000,-0.867518
labor_force,0.591484,0.981657,-0.989861,0.097192,-0.867518,1.000000


Since literacy_rate and labor_force has high correlation, i won't write one of them in the next model.

In [16]:
model = ols('mortality_rate ~ literacy_rate + poverty_hc_ratio + c_health_exp + gdp', data = df).fit()
model.summary() 

c:\Users\duygu\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         mortality_rate   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     118.5
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           3.75e-07
Time:                        18:03:52   Log-Likelihood:                -12.445
No. Observations:                  13   AIC:                             34.89
Df Residuals:                       8   BIC:                             37.71
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           80.8631     12.126      6.669      0.000      52.901     108.825
literacy_rate       -0.9539      0.103     -9.227      0.000      -1.192      -0.715
poverty_hc_ratio     0.9820      0.326      3.013      0.017       0.230       1.734
c_health_exp        -0.0177      0.009     -1.864      0.099      -0.040       0.004
gdp                  0.0013      0.001      2.674      0.028       0.000       0.003
==============================================================================
Omnibus:                        0.566   Durbin-Watson:                   2.136
Prob(Omnibus):                  0.754   Jarque-Bera (JB):                0.403
Skew:                          -0.374   Prob(JB):                        0.818
Kurtosis:                       2.572   Cond. No.                     5.71e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.71e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Let's focus on the p-values. The p values are less than 0.05 except c_health_exp. This can show us other independent variables have a significant impact on our dependent variable prediction, mortality rate in this case.

In [17]:
anova_result = sm.stats.anova_lm(model)
anova_result

,df,sum_sq,mean_sq,F,PR(>F)
literacy_rate,1.0,296.579278,296.579278,459.459949,2.361757e-08
poverty_hc_ratio,1.0,3.560728,3.560728,5.516272,4.677795e-02
c_health_exp,1.0,1.161955,1.161955,1.800098,2.165360e-01
gdp,1.0,4.614076,4.614076,7.148116,2.820458e-02
Residual,8.0,5.163963,0.645495,NaN,NaN


At the end last model seems best one with low p-value and high R^2-score. We have used 4 independent variable instead of 5. Because there was a high correlation between two of them.

# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

In [18]:
import yfinance as yf

In [19]:
gf = yf.download('GC=F')
sf = yf.download('SI=F')
cf = yf.download('HG=F')
pf = yf.download('PL=F')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
gf

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,273.899994,273.899994,273.899994,0
2000-08-31 00:00:00-04:00,274.799988,278.299988,274.799988,278.299988,278.299988,0
2000-09-01 00:00:00-04:00,277.000000,277.000000,277.000000,277.000000,277.000000,0
2000-09-05 00:00:00-04:00,275.799988,275.799988,275.799988,275.799988,275.799988,2
2000-09-06 00:00:00-04:00,274.200012,274.200012,274.200012,274.200012,274.200012,0
...,...,...,...,...,...,...
2022-11-01 00:00:00-04:00,1630.800049,1653.699951,1630.800049,1645.000000,1645.000000,985
2022-11-02 00:00:00-04:00,1650.800049,1664.699951,1634.000000,1645.699951,1645.699951,612
2022-11-03 00:00:00-04:00,1629.199951,1629.199951,1615.099976,1627.300049,1627.300049,1427


In [21]:
tmp = {}

tmp['gf_open'] = gf['Open']
tmp['sf_open'] = sf['Open']
tmp['cf_open'] = cf['Open']
tmp['pf_open'] = pf['Open']

tmp['gf_close'] = gf['Close']
tmp['sf_close'] = sf['Close']
tmp['cf_close'] = cf['Close']
tmp['pf_close'] = pf['Close']

data = pd.DataFrame(tmp).dropna()
data

,gf_open,sf_open,cf_open,pf_open,gf_close,sf_close,cf_close,pf_close
Date,,,,,,,,
2000-08-30 00:00:00-04:00,273.899994,4.950000,0.8790,593.900024,273.899994,4.930000,0.8850,591.400024
2000-08-31 00:00:00-04:00,274.799988,4.920000,0.8850,589.000000,278.299988,5.003000,0.8850,586.700012
2000-09-01 00:00:00-04:00,277.000000,5.035000,0.8780,588.000000,277.000000,5.004000,0.8890,595.299988
2000-09-05 00:00:00-04:00,275.799988,4.990000,0.8960,602.000000,275.799988,4.998000,0.9060,601.299988
2000-09-06 00:00:00-04:00,274.200012,5.000000,0.9050,603.000000,274.200012,4.983000,0.9015,611.099976
...,...,...,...,...,...,...,...,...
2022-11-01 00:00:00-04:00,1630.800049,19.125000,3.4945,959.799988,1645.000000,19.673000,3.5095,959.799988
2022-11-02 00:00:00-04:00,1650.800049,19.780001,3.4985,960.200012,1645.699951,19.600000,3.5055,960.200012
2022-11-03 00:00:00-04:00,1629.199951,19.235001,3.4455,933.400024,1627.300049,19.436001,3.4565,933.400024


When we put opening prices of gold in our model, we are getting R^2-score as 1. So it is better not to use gold open data for our model, that is why I haven't added to our model.

In [22]:
X = data[['sf_open', 'cf_open','pf_open']]     
XX = sm.add_constant(X)                   
Y = data['gf_close']
model = sm.OLS(Y,XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               gf_close   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                 1.168e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:05:14   Log-Likelihood:                -32183.
No. Observations:                4865   AIC:                         6.437e+04
Df Residuals:                    4861   BIC:                         6.440e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        431.4798      8.453     51.046      0.0

In [23]:
X = data[['gf_open', 'cf_open','pf_open']]    
XX = sm.add_constant(X)                     
Y = data['sf_close']
model = sm.OLS(Y,XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               sf_close   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:05:17   Log-Likelihood:                -11843.
No. Observations:                4865   AIC:                         2.369e+04
Df Residuals:                    4861   BIC:                         2.372e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.5913      0.129    -51.009      0.0

In [24]:
X = data[['gf_open', 'sf_open', 'pf_open']]    
XX = sm.add_constant(X)                     
Y = data['cf_close']
model = sm.OLS(Y,XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               cf_close   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     8061.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:05:21   Log-Likelihood:                -3073.2
No. Observations:                4865   AIC:                             6154.
Df Residuals:                    4861   BIC:                             6180.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3248      0.026    -12.494      0.0

In [25]:
X = data[['gf_open', 'cf_open', 'sf_open']]    
XX = sm.add_constant(X)                     
Y = data['pf_close']
model = sm.OLS(Y,XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               pf_close   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     5763.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:05:24   Log-Likelihood:                -31933.
No. Observations:                4865   AIC:                         6.387e+04
Df Residuals:                    4861   BIC:                         6.390e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        524.5003      6.521     80.435      0.0

In [27]:
model = ols('sf_close ~ gf_open + cf_open + pf_open', data = data).fit()
model.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               sf_close   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        18:05:41   Log-Likelihood:                -11843.
No. Observations:                4865   AIC:                         2.369e+04
Df Residuals:                    4861   BIC:                         2.372e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.5913      0.129    -51.009      0.000      -6.845      -6.338
gf_open        0.0115      0.000     79.593      0.000       0.011       0.012
cf_open       -0.4500      0.087     -5.173      0.000      -0.620      -0.279
pf_open        0.0109      0.000     63.845      0.000       0.011       0.011
==============================================================================
Omnibus:                     1300.018   Durbin-Watson:                   0.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6397.530
Skew:                           1.200   Prob(JB):                         0.00
Kurtosis:                       8.080   Cond. No.                     5.44e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
gf_open,1.0,275715.830251,275715.830251,36152.071429,0.0
cf_open,1.0,20673.505522,20673.505522,2710.725923,0.0
pf_open,1.0,31086.864928,31086.864928,4076.133606,0.0
Residual,4861.0,37072.693150,7.626557,NaN,NaN


Our best model seems like last one which has 0.898 R^2 - score. If R^2 is 1 that shows us dependent variable can be perfectly explained without error by independent variables. 
So it is better if our R^2 is closer to 1, which almost 0.9 in this case.

# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [29]:
gf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,273.899994,273.899994,273.899994,0
2000-08-31 00:00:00-04:00,274.799988,278.299988,274.799988,278.299988,278.299988,0
2000-09-01 00:00:00-04:00,277.000000,277.000000,277.000000,277.000000,277.000000,0
2000-09-05 00:00:00-04:00,275.799988,275.799988,275.799988,275.799988,275.799988,2
2000-09-06 00:00:00-04:00,274.200012,274.200012,274.200012,274.200012,274.200012,0


In [30]:
def ticker(ticker_symbol):
    df = pd.DataFrame(columns = ["Price"])
    for index, row in ticker_symbol.iterrows():
        if row["Close"] - row["Open"] >= 0:
            df.loc[index] = [1]
        else:
            df.loc[index] = [0]
    return df

# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [33]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header = None)

fn = {'+': 1, '-': 0}

X = credit.replace('?', 0).iloc[:, [1, 2, 7, 10, 14]]
y = credit.iloc[:, 15].map(lambda x: fn.get(x,0))
credit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [34]:
X

,1,2,7,10,14
0,30.83,0.000,1.25,1,0
1,58.67,4.460,3.04,6,560
2,24.50,0.500,1.50,0,824
3,27.83,1.540,3.75,5,3
4,20.17,5.625,1.71,0,0
...,...,...,...,...,...
685,21.08,10.085,1.25,0,0
686,22.67,0.750,2.00,2,394
687,25.25,13.500,2.00,1,1
688,17.92,0.205,0.04,0,750


In [35]:
y

0      1
1      1
2      1
3      1
4      1
      ..
685    0
686    0
687    0
688    0
689    0
Name: 15, Length: 690, dtype: int64

In [36]:
credit.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

In [37]:
credit.columns = credit.columns.astype(str)
credit.columns.map(type)

Index([<class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>,
       <class 'str'>, <class 'str'>, <class 'str'>, <class 'str'>],
      dtype='object')

In [38]:
credit["15"].value_counts()

-    383
+    307
Name: 15, dtype: int64

We have checked the distribution of out target variable y:
It is good that "+" and "-" values are near to each other. 

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)

model = LogisticRegression(max_iter=1500)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
y_predict = model.predict(X_test)
confusion_matrix(y_test,y_predict)

0.7803468208092486


array([[90,  4],
       [34, 45]], dtype=int64)

In [46]:
def bootstrap(X,y,model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [47]:
model = LogisticRegression(max_iter=1500)
bootstrap(X,y,model)

(0.6994219653179191, 0.7976878612716763)

In [51]:
model = LinearRegression()
bootstrap(X,y,model)

(-0.049528928302915, 0.2739965465251252)

In [54]:
model = DecisionTreeRegressor()
bootstrap(X,y,model)

(-0.44561643835616427, -0.017375033413525642)

Now let's try some logistic regression solvers. There are some solvers which is ready and these are {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}. If we do not write anything, it will take ’lbfgs’ by default. To try these solvers, first we will fit the model and then try some solvers. I won't write all of them because some solvers are giving the same result. After fitting the model, we will predict class labels on a test data. After printing intercept and slope values, we will try to get accuracy of model with using score method.

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

model = LogisticRegression(solver='sag', max_iter = 7000)
clf = model.fit(X_train, y_train) # We call our estimator instance clf, as it is a classifier.

In [72]:
LR1_pred_labels = model.predict(X_test)

print('Intercept (Beta 0): ', clf.intercept_)
print('Slope (Beta 1): ', clf.coef_)

Intercept (Beta 0):  [-0.0003807]
Slope (Beta 1):  [[-0.00632571  0.00045778  0.0015603   0.00294327  0.00050595]]


In [73]:
score = model.score(X_test, y_test)
print('Accuracy Score: ', score)

Accuracy Score:  0.6811594202898551


In [74]:
print(classification_report(y_test, LR1_pred_labels))

              precision    recall  f1-score   support

           0       0.67      0.87      0.76       117
           1       0.72      0.43      0.54        90

    accuracy                           0.68       207
   macro avg       0.69      0.65      0.65       207
weighted avg       0.69      0.68      0.66       207



Our accuracy score (Correct predictions / Total predictions) is 0.68 which is not good and when we look at classification report, we can see that our model is not enough. Even tossed coins have 0.50 chance.

Precision part shows us (True Positives / (True Positives + False Positives)) and if it is low that means we have higher number of False Positives.

Recall (True Positives / (True Positives + False Negatives)) The ability of a model to find all the relevant cases within a data set

F1-score = Average between Precision and Recall (weights can be applied if one metric is more important than the other for a specific use case)

Support = Number of actual observations in that class

So we can basically say that there are also lots of cases (32%) where our model can not predict the outcome successfully. So let's try something else

In [75]:
model = LogisticRegression(solver='lbfgs')
clf = model.fit(X_train, y_train)

LR2_pred_labels = model.predict(X_test)

print('Intercept (Beta 0): ', clf.intercept_)
print('Slopes (Beta 1 and Beta 2): ', clf.coef_)

Intercept (Beta 0):  [-1.85101208]
Slopes (Beta 1 and Beta 2):  [[0.00524796 0.04707754 0.2063027  0.34437268 0.00040689]]


In [76]:
score = model.score(X_test, y_test)
print('Accuracy Score: ', score)

print(classification_report(y_test, LR2_pred_labels))

Accuracy Score:  0.7342995169082126
              precision    recall  f1-score   support

           0       0.71      0.89      0.79       117
           1       0.79      0.53      0.64        90

    accuracy                           0.73       207
   macro avg       0.75      0.71      0.71       207
weighted avg       0.74      0.73      0.72       207



When we look at classification report to evaulate our model, we can see that it is better than before.
Even if our model is not great again, it still helps us to identify y in 73% of the cases, which is better than a random guess.

In [79]:
model = LogisticRegression(max_iter=2000,solver='liblinear')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[104,  13],
       [ 42,  48]], dtype=int64)

Confusion matrix helps us to determine how often the model prediction is correct (accuracy of the model)

The result is telling us that we have 104 + 48 correct predictions and 13 + 42 incorrect predictions.

( TP + TN ) / Total = 104 + 48 / 207 = 0.73

This means that the model is 73% correct. While there is 27% error in the model.